In [359]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geojson import GeometryCollection
import geojson

In [360]:
# Point data to geoJSON
import random

df_red = pd.read_csv("redlist_assessments.csv")

# Assume: RedList scientific correspond to UBC species
red_sciNames = df_red.scientificName

def some(x, n):
    return x.loc[random.sample(x.index, n)]

def pointCSVtoJSON(filename, num_rows=0):
    df = pd.read_csv(filename, delimiter="\t", low_memory=False)
    if (num_rows):
        df = some(df, num_rows)
    geometry = [Point(xy) for xy in zip (df['decimalLongitude'], df['decimalLatitude'])]
    geo_df = gpd.GeoDataFrame(df, geometry = geometry, crs = {'init': 'epsg:4326'})
    
    # gbif_geo_df = gbif_geo_df[gbif_geo_df['species'].isin(red_sciNames)]
    geo_df['redList'] = geo_df.apply(lambda x: int(x['species']in red_sciNames.values), 1).values

    geo_df.to_file("leaflet/" + filename.split('.')[0] + '.geojson', driver="GeoJSON")
    return geo_df

In [361]:
gbif_geo_df = pointCSVtoJSON('gbif_tot.csv', 2000)

In [335]:
# gbif_geo_df.to_file("leaflet/" +'red.geojson', driver="GeoJSON")

In [307]:
# SHP data to geoJSON
def fix_crs(map_ob):
    return map_ob.to_crs({'init': 'epsg:4326'})
str_map = gpd.read_file("ecological_reserves/BC_Eco_Reserves.shp")
str_map = fix_crs(str_map)

In [308]:
# # GeoJSON does not support multipolygon. Doesn't work
# str_map.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")

# # fiona doesn't work
# import fiona
# import json

# with fiona.open('ecological_reserves/BC_Eco_Reserves.shp') as source:
#     records = list(source)
# geo_json = {"type": "FeatureCollection","features": records}
# with open('leaflet/UBC_poly.geojson', 'w') as fp:
#     json.dump(geo_json, fp)

In [309]:
# Convert multipolygon to single polygons

In [310]:
geom_series = str_map.geometry

In [311]:
def geom_apply(x):
    try:
        return list(x)
    except TypeError:
        return [x]

In [312]:
geom_series = geom_series.apply(geom_apply)

In [313]:
geom_series = geom_series.apply(pd.Series).stack()

In [314]:
# # Quick but loses properties
# list_poly = list(geom_series)
# geoms = GeometryCollection(list_poly)
# geo_file = geojson.dumps(geoms)
# with open("leaflet/UBC_poly.geojson", "w") as text_file:
#     text_file.write(geo_file)

In [315]:
# Careful! Deep copy required here to avoid chaining

df = pd.DataFrame(columns=str_map.columns)
for ind, poly in geom_series.iteritems():
    curr_row = str_map.loc[ind[0]].copy(deep=True)
    curr_row['geometry'] = poly
    df = df.append(curr_row)

df_gpd = gpd.GeoDataFrame(df,geometry = df.geometry, crs = {'init': 'epsg:4326'})
df_gpd.to_file("leaflet/UBC_poly.geojson", driver="GeoJSON")